# PCA Anomalies

In [ ]:
include("stuff.jl")

In [ ]:
cube_base = getCubeData(Cube(data_cube_dir), variable = vars, time = (Date("2001-01-01"), Date("2011-12-31")))
pca_no_msc_cube = loadCube(joinpath(cubes_base_dir, "pca_no_msc_cube"))
pca_msc_cube = loadCube(joinpath(cubes_base_dir, "pca_msc_cube"))
cube_time_space_mask = loadCube(joinpath(cubes_base_dir, "cube_time_space_mask"))
cube_msc_time_space_mask = loadCube(joinpath(cubes_base_dir, "cube_msc_time_space_mask"))
cube_pca_online = loadCube(joinpath(cubes_base_dir, "cube_pca_online"))

In [ ]:
cube_z_trans = load(joinpath(cubes_base_dir, "z_trans_pca.jld2"), "z_trans")
cube_pca = load(joinpath(cubes_base_dir, "z_trans_pca.jld2"), "pca")

In [ ]:
cube_extrema_msc = loadCube(joinpath(cubes_base_dir, "cube_extrema_msc"))
cube_extrema_no_msc = loadCube(joinpath(cubes_base_dir, "cube_extrema_no_msc"))
n_seasons = 4
comp = 1
doy_idxs = (length(getAxis(MSCAxis, pca_msc_cube)) ÷ n_seasons) .* (0:n_seasons - 1) .+ 1 |> collect
ex_msc = [
    cube_extrema_msc[:, i] |> 
    x -> abs.(x) |> 
    x -> max(x...) |> 
    x -> (-x, x) 
    for i in 1:size(cube_extrema_msc, 2)
]
ex_no_msc = [
    cube_extrema_no_msc[:, i] |> 
    x -> abs.(x) |>
    x -> max(x...) |> 
    x -> (-x, x) 
    for i in 1:size(cube_extrema_no_msc, 2)
]

In [ ]:
places

In [ ]:
pc1_msc = Float64[]
pc2_msc = Float64[]
pc3_msc = Float64[]
l_msc = String[]

for i in 1:length(places[1])
    append!(pc1_msc, pca_msc_cube[:, 1, places[1][i], size(pca_msc_cube, 4) - places[2][i] + 1])
    append!(pc2_msc, pca_msc_cube[:, 2, places[1][i], size(pca_msc_cube, 4) - places[2][i] + 1])
    append!(pc3_msc, pca_msc_cube[:, 3, places[1][i], size(pca_msc_cube, 4) - places[2][i] + 1])
    append!(l_msc, repeat([ places[3][i] ], size(pca_msc_cube, 1)))
end

pc1 = Float64[]
pc2 = Float64[]
pc3 = Float64[]
l = String[]
for i in 1:length(places[1])
    append!(pc1, cube_pca_online[1, places[1][i], size(cube_pca_online, 3) - places[2][i] + 1, :] |> collect |> missing_to_nan)
    append!(pc2, cube_pca_online[2, places[1][i], size(cube_pca_online, 3) - places[2][i] + 1, :] |> collect |> missing_to_nan)
    append!(pc3, cube_pca_online[3, places[1][i], size(cube_pca_online, 3) - places[2][i] + 1, :] |> collect |> missing_to_nan)
    append!(l, repeat([ places[3][i] ], size(cube_pca_online, 4)))
end

In [ ]:
using DelimitedFiles

In [ ]:
writedlm(
    "location_table.csv", 
    [    
        ["time" "PC1" "PC2" "PC3" "Location"];
        repeat(getAxis("Time", cube_base).values, 6) [pc1    pc2   pc3   l]
    ]
    
)


In [ ]:
function get_msc_hist(data, mask, d1, d2, places)
    valid_value_idx_msc = mask[:, :, :] .== 1

    x_all_msc = vec(data[:, d1, :, :])[vec(valid_value_idx_msc)] |> missing_to_nan
    y_all_msc = vec(data[:, d2, :, :])[vec(valid_value_idx_msc)] |> missing_to_nan
    w_all_msc = cosd.(repeat(
            collect(getAxis(LatAxis, data).values), 
            # lat comes last:
            inner = size(data, 1) * size(data, 3)
        )[vec(valid_value_idx_msc)]
    )

    fit(
        Histogram, 
        (x_all_msc, y_all_msc), 
        StatsBase.weights(w_all_msc), 
        #(-8.5f0:0.25f0:4.0f0, -5.0f0:0.25f0:7.5f0),
        #(-5.5f0:0.5f0:9.5f0, -8.0f0:0.5f0:7.0f0),
        closed = :right
    ) |> LinearAlgebra.normalize
end



function get_hist(data, mask, d1, d2, places)
    valid_value_idx = mask[:, :, :] .== 1

    x_all = vec(data[d1, :, :, :])[vec(valid_value_idx)] |> missing_to_nan
    y_all = vec(data[d2, :, :, :])[vec(valid_value_idx)] |> missing_to_nan

    w_all = cosd.(repeat(
            collect(getAxis(LatAxis, data).values), 
            inner = size(data, 2),
            outer = size(data, 4)
        )[vec(valid_value_idx)]
    )

    fit(
        Histogram,
        (x_all, y_all),
        StatsBase.weights(w_all), 
        #(-4.5f0:0.5f0:9.5f0, -8.0f0:0.5f0:7.0f0); 
        closed = :right
    ) |> LinearAlgebra.normalize
end    

In [ ]:
@time hist_msc_12 = get_msc_hist(pca_msc_cube, cube_msc_time_space_mask, 1, 2, places)
@time hist_msc_13 = get_msc_hist(pca_msc_cube, cube_msc_time_space_mask, 1, 3, places)
@time hist_msc_23 = get_msc_hist(pca_msc_cube, cube_msc_time_space_mask, 2, 3, places)

@time hist_12 = get_hist(cube_pca_online, cube_time_space_mask, 1, 2, places)
@time hist_13 = get_hist(cube_pca_online, cube_time_space_mask, 1, 3, places)
@time hist_23 = get_hist(cube_pca_online, cube_time_space_mask, 2, 3, places)

In [ ]:
    # indexing in the cube starts in the NORTH WEST corner, see the direction of the axes for details!!!

pca_msc_cube_places = [pca_msc_cube[:, :, places[1][i], 720 - places[2][i]] for i in 1:length(places[1])]
pca_msc_cube_places = cat(pca_msc_cube_places..., dims = 3)
pca_msc_cube_places = convert(Array{Float64}, pca_msc_cube_places)

In [ ]:
    # indexing in the cube starts in the NORTH WEST corner, see the direction of the axes for details!!!

cube_base_places = [cube_base[places[1][i], 720 - places[2][i], :, :] for i in 1:length(places[1])]
cube_base_places = cat(cube_base_places..., dims = 3)

In [ ]:
R"""
pdf($(joinpath(fig_path, "distr_traj.pdf")), width = 8, height = 12)

# colors for the lines:
palette(brewer.pal(name = "Set1", n = 6))

# layout
wds <- c(0.15, 1, 1)
hgs <- rep(c(1, 0.1), 3)
line_gap <- 0.05
layout(matrix(c(4, 1, 2,
                0, 3, 3,
                8, 5, 6,
                0, 7, 7,
               12, 9, 10,
                0, 11, 11), 
              nrow = 6, ncol = 3, byrow = TRUE), 
       widths = wds, heights = hgs)

max_dens <- $(max(maximum(hist_12.weights), maximum(hist_msc_12.weights),
                  maximum(hist_13.weights), maximum(hist_msc_13.weights),
                  maximum(hist_23.weights), maximum(hist_msc_23.weights)))




# to show the actual extent of the distribution, 
# we remove the whites from the palette and set 
# only real zeros to white and small values to the 
# lightest grey value

pal <- brewer.pal(name = "Greys", n = 9)
# remove white from the palette
pal <- pal[-1]
# pal <- colorRampPalette(colors = pal)(100)

plot_dens <- function(d, x_ax, y_ax, color_pal = pal, yaxt = "s") {
    d[d == 0] <- NA
    image(x = x_ax, y = y_ax, d, col = color_pal, bty = "n", yaxt = yaxt, xlab = NA, ylab = NA)
}

plot_traj <- function(df, is_msc = FALSE) {
    for (l in 1:nlevels(df$l)) {
        lines(df[as.numeric(df$l) == l,], col = l, lwd = if(is_msc) 4 else 0.1)
    }    
}

draw_arrow <- function(df, country, center = floor(46 / 4)) {
    c_level <- which(levels(df$l) == country)
    df[as.numeric(df$l) == c_level,] %>%
        {.[c(center, center + 1), ]} %>%
        {
            arrows(
                .$x[1], .$y[1], .$x[2], .$y[2], 
                lwd = 4, length = 0.15,
                col = c_level
            )
        }
}


center_line <- function(h = 1) {
    grid.lines(
        x = (wds[1] + wds[2]) / sum(wds),
        y = c(sum(hgs[(2 * h    ):6]) / sum(hgs) + line_gap, 
              sum(hgs[(2 * h - 1):6]) / sum(hgs) - line_gap)
    )    
}

plot_map <- function(fig) {
    par(new = TRUE, fig = fig, mar = c(0, 0, 0, 0))
    plot(1:10, type = "n", xlim = c(-180, 180), ylim = c(-60, 90), axes = F)
    maps::map("world", col = "black", ylim=c(-60, 90), add = TRUE)

    order_idx <- order($(places[3]))

    points(
        $(cube_base.lonAxis.values[places[1]])[order_idx],
        $(-cube_base.latAxis.values[places[2]])[order_idx], 
        col = seq_along(order_idx),
        pch = 16, cex = 3
    )
}

get_mod_46_plus_t <- function(t, lim) {
    seq(from = t, to = lim, by = 46)
}

plot_example_values <- function(x, y, t, comp1, comp2, place, adj = c(0, 0)) {
    vars <- c("gross_primary_productivity", "terrestrial_ecosystem_respiration", 
              "sensible_heat", "latent_energy", "root_moisture", 
              "black_sky_albedo")
    var_labels <- c("GPP", "Ecosystem Respiration",
                    "Sensible Heat", "Latent Heat", "Root Moisture",
                    "Black Sky Albedo")
    cube_var_idx <- sapply(vars, function(v) which($(getAxis(VariableAxis, cube_base).values) %in% v))

    var_vals <- $(cube_base_places)[get_mod_46_plus_t(t, 506), cube_var_idx, place]
    var_vals <- colMeans(var_vals, na.rm = TRUE)
    comp1_val <- $(pca_msc_cube_places)[t, comp1, place]
    comp2_val <- $(pca_msc_cube_places)[t, comp2, place]

    lab <- mapply(
        function(l, v) paste0(l, ": ", sprintf("%0.2f", v)), 
        l = var_labels, v = var_vals
    )

    segments(comp1_val, comp2_val, x, y)

    text(x = x, y = y, labels = paste(lab, collapse = "\n"), adj = adj) 
}


## 1-2 #########################################################################
par(mar = c(2.1, 0, 0, 0.4))
par(las = 1)


plot_dens(
    $(hist_12.weights), 
    $(collect(hist_12.edges[1])),
    $(collect(hist_12.edges[2]))
)


plot_traj(data.frame(x = $pc1, y = $pc2, l = factor($l)))


plot_dens(
    $(hist_msc_12.weights),
    $(collect(hist_msc_12.edges[1])),
    $(collect(hist_msc_12.edges[2])),
    yaxt = "n"
)

df <- data.frame(x = $pc1_msc, y = $pc2_msc, l = factor($l_msc))
plot_traj(df, is_msc = TRUE)
draw_arrow(df, "Mexico")
draw_arrow(df, "Moscow")
draw_arrow(df, "India", center = 46 / 2)
draw_arrow(df, "Siberia", center = 19)

par(xpd = NA)
plot_example_values(x = -4, y =  1, t = 22, comp1 = 1, comp2 = 2, place = 1, adj = c(1, 0.5))
plot_example_values(x = -2, y = -5, t =  5, comp1 = 1, comp2 = 2, place = 3, adj = c(1, 1))
plot_example_values(x =  4, y = -4, t = 40, comp1 = 1, comp2 = 2, place = 4, adj = c(1, 1))
par(xpd = FALSE)

legend(
    "topright",
    legend = levels(df$l),
    col = seq_along(levels(df$l)),
    lty = 1,
    lwd = 7,
    pch = 1,
    pt.cex = 2,
    cex = 1.2,
    bty = 'n'
)

plot.new()
mtext(expression(low~productivity%<-%PC[1]%->%high~productivity), 1)

par(mar = c(0, 2, 0, 0))
plot.new()
mtext(expression(wet%<-%PC[2]%->%dry), 2, las = 0)


## 1-3 #########################################################################
par(mar = c(2.1, 0, 0, 0.4))
par(las = 1)

plot_dens(
    $(hist_13.weights), 
    $(collect(hist_13.edges[1])),
    $(collect(hist_13.edges[2]))
)

plot_traj(data.frame(x = $pc1, y = $pc3, l = factor($l)))

plot_dens(
    $(hist_msc_13.weights),
    $(collect(hist_msc_13.edges[1])),
    $(collect(hist_msc_13.edges[2])),
    yaxt = "n"
)

df <- data.frame(x = $pc1_msc, y = $pc3_msc, l = factor($l_msc))

plot_traj(df, is_msc = TRUE)
draw_arrow(df, "Mexico")
draw_arrow(df, "Moscow")
draw_arrow(df, "India", center = 46 / 2)
draw_arrow(df, "Siberia", center = 19)

par(xpd = NA)
plot_example_values(x = -2, y = -3, t =  5, comp1 = 1, comp2 = 3, place = 2, adj = c(1, 0.5))
plot_example_values(x = -6, y =  4, t =  6, comp1 = 1, comp2 = 3, place = 3, adj = c(1, 0.5))
plot_example_values(x =  6, y = -1, t = 40, comp1 = 1, comp2 = 3, place = 4, adj = c(1, 1))
par(xpd = FALSE)

plot.new()
mtext(expression(low~productivity%<-%PC[1]%->%high~productivity), 1)

par(mar = c(0, 2, 0, 0))
plot.new()
mtext(expression(low~albedo%<-%PC[3]%->%high~albedo), 2, las = 0)

## 2-3 #########################################################################
par(mar = c(2.1, 0, 0, 0.4))
par(las = 1)

plot_dens(
    $(hist_23.weights), 
    $(collect(hist_23.edges[1])),
    $(collect(hist_23.edges[2]))
)

plot_traj(data.frame(x = $pc2, y = $pc3, l = factor($l)))

plot_dens(
    $(hist_msc_23.weights),
    $(collect(hist_msc_23.edges[1])),
    $(collect(hist_msc_23.edges[2])),
    yaxt = "n"
)

df <- data.frame(x = $pc2_msc, y = $pc3_msc, l = factor($l_msc))

plot_traj(df, is_msc = TRUE)
draw_arrow(df,"Mexico")
draw_arrow(df,"Moscow", center = 20)
draw_arrow(df,"India", center = 46 / 2)
draw_arrow(df,"Siberia", center = 19)

par(xpd = NA)
plot_example_values(x = 3, y =  4, t = 22, comp1 = 2, comp2 = 3, place = 1, adj = c(1, 0.5))
plot_example_values(x = -5, y = 4, t =  5, comp1 = 2, comp2 = 3, place = 3, adj = c(1, 1))
plot_example_values(x = -4, y = -3, t =  5, comp1 = 2, comp2 = 3, place = 2, adj = c(1, 0.5))
par(xpd = FALSE)

plot.new()
mtext(expression(wet%<-%PC[2]%->%dry), 1, las = 0)

par(mar = c(0, 2, 0, 0))
plot.new()
mtext(expression(low~albedo%<-%PC[3]%->%high~albedo), 2, las = 0)


############################### decorations ##################


center_line(1)
center_line(2)
center_line(3)

label_grid <- function(l, x, y) {
    grid.text(
        l, 
        (sum(wds[1:x]) + line_gap / 2) / sum(wds),
        (1 - (y - 1) / 3) - line_gap * hgs[1] / sum(hgs),
        just = c(0, 1)
    )
}

label_grid("(a)", 1, 1)
label_grid("(b)", 2, 1)
label_grid("(c)", 1, 2)
label_grid("(d)", 2, 2)
label_grid("(e)", 1, 3)
label_grid("(f)", 2, 3)

plot_map(fig = c(0.18, 0.5, 0.72, 0.85))

dev.off()

"""

In [ ]:
show_pdf(
    "http://127.0.0.1:8333/files/results/fig/distr_traj.pdf",
    width = 700,
    height = 1100
)

In [ ]:
cube_base

rmprocs(workers())
addprocs(40)
cube_mask = mapCube(
    cube_base,
    indims = InDims(VariableAxis),
    outdims = OutDims()
) do xout, xin
    xout[1] = !any(ismissing, xin)
end
rmprocs(workers())

n = cube_mask[:, :, :] |> skipmissing |> x -> Int64.(x) |> sum

720 * 1440 * 506

n / (720 * 1440 * 506)

In [ ]:
# California

pc1, t_idx = pca_msc_cube[:,     1, places[1][1], 720 - places[2][1]] |> findmin
pc2        = pca_msc_cube[t_idx, 2, places[1][1], 720 - places[2][1]] 
cal_xy = [pc1, pc2]
cal_var = [ 
    cube_base[places[1][1], 720 - places[2][1], t_idx,  5],
    cube_base[places[1][1], 720 - places[2][1], t_idx, 11],
    cube_base[places[1][1], 720 - places[2][1], t_idx,  6],
    cube_base[places[1][1], 720 - places[2][1], t_idx,  9],
    cube_base[places[1][1], 720 - places[2][1], t_idx,  8] 
]

# Siberia
pc2, t_idx = pca_msc_cube[:,     2, places[1][3], 720 - places[2][3]] |> findmin
pc1        = pca_msc_cube[t_idx, 1, places[1][3], 720 - places[2][3]] 
sib_xy = [pc1, pc2]
sib_var = [ 
    cube_base[places[1][3], 720 - places[2][3], t_idx,  5],
    cube_base[places[1][3], 720 - places[2][3], t_idx, 11],
    cube_base[places[1][3], 720 - places[2][3], t_idx,  9],
    cube_base[places[1][3], 720 - places[2][3], t_idx,  6],
    cube_base[places[1][3], 720 - places[2][3], t_idx,  8] 
]


# Amazon
pc1, t_idx = pca_msc_cube[:,     1, places[1][4], 720 - places[2][4]] |> findmax
pc2        = pca_msc_cube[t_idx, 2, places[1][4], 720 - places[2][4]] 
ama_xy = [pc1, pc2]
ama_var = [ 
    cube_base[places[1][4], 720 - places[2][4], t_idx,  5],
    cube_base[places[1][4], 720 - places[2][4], t_idx, 11],
    cube_base[places[1][4], 720 - places[2][4], t_idx,  9],
    cube_base[places[1][4], 720 - places[2][4], t_idx,  6],
    cube_base[places[1][4], 720 - places[2][4], t_idx,  8] 
]


In [ ]:
rmprocs(procs())
addprocs(40)
cube_max = mapCube(
    cube_base,
    indims = InDims("Time"),
    outdims = OutDims()
) do xout, xin
    xout[1] = findmax(collect(skipmissing(xin)))[1]
end
rmprocs(procs())

In [ ]:
rmprocs(procs())
addprocs(40)
cube_min = mapCube(
    cube_base,
    indims = InDims("Time"),
    outdims = OutDims()
) do xout, xin
    xout[1] = findmin(collect(skipmissing(xin)))[1]
end
rmprocs(procs())

In [ ]:
rmprocs(procs())
addprocs(40)
msc_cube = getMSC(cube_base)
rmprocs(procs())

In [ ]:
msc_cube

In [ ]:
rmprocs(procs())
addprocs(40)
cube_max_msc = mapCube(
    msc_cube,
    indims = InDims("MSC"),
    outdims = OutDims()
) do xout, xin
    xout[1] = findmax(collect(skipmissing(xin)))[1]
end
rmprocs(procs())

In [ ]:
rmprocs(procs())
addprocs(40)
cube_min_msc = mapCube(
    msc_cube,
    indims = InDims("MSC"),
    outdims = OutDims()
) do xout, xin
    xout[1] = findmin(collect(skipmissing(xin)))[1]
end
rmprocs(procs())

In [ ]:
plotMAP(cube_max)

In [ ]:
plotMAP(cube_min)

In [ ]:
plotMAP(cube_max_msc)

In [ ]:
plotMAP(cube_min_msc)

In [ ]:
R"""

image(x = 1:1440, y =  1:720, 
      z = $(cube_base[:, :, 1, 1] |> collect |> missing_to_nan |> x -> x[:, end:-1:1] .|> Float64))
points($(places[1]), $(places[2]))

"""

In [ ]:
cube_base = getCubeData(Cube(data_cube_dir), variable = vars, time = (Date("2001-01-01"), Date("2011-12-31")))

In [ ]:
function recon_pc12(pc1, pc2, idx)
    [pc1, pc2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] |>
        x -> reconstruct(cube_pca, x)                            |>
        x -> (x .* cube_z_trans.scale) .+ cube_z_trans.mean      |>
        x -> x[idx]
end

In [ ]:
recon_le = recon_pc12.(collect(-4:0.25:5), permutedims(collect(-5:0.25:4)), 6)
recon_h = recon_pc12.(collect(-4:0.25:5), permutedims(collect(-5:0.25:4)), 9)
recon_bowen = recon_h ./ recon_le

In [ ]:
R"""
pdf($(joinpath(fig_path, "bowen.pdf")), width = 9, height = 3.3)

layout(t(1:3))
par(mar = c(4, 4, 0, 0) + 0.1)

pal <- brewer.pal(name = "Greys", n = 9)
x_coord <- seq(-4, 5, by = 0.25)
y_coord <- seq(-5, 4, by = 0.25)
w_msc <- $(hist_msc_12.weights)
max_dens <- max(w_msc)
image(
    x = $(collect(hist_msc_12.edges[1])),
    y = $(collect(hist_msc_12.edges[2])),
    w_msc, 
    #useRaster = TRUE, 
    col = pal,
    bty = "n",
    #yaxt = "n",
    xlim = range(x_coord),
    ylim = range(y_coord),
    zlim = c(0, max_dens),
    xlab = expression(PC[1]), ylab = expression(PC[2]),
    asp = 1
)
contour(x_coord, y_coord, $recon_le, add = TRUE)
image(
    x = $(collect(hist_msc_12.edges[1])),
    y = $(collect(hist_msc_12.edges[2])),
    w_msc, 
    #useRaster = TRUE, 
    col = pal,
    bty = "n",
    yaxt = "n",
    xlim = range(x_coord),
    ylim = range(y_coord),
    zlim = c(0, max_dens),
    xlab = expression(PC[1]), ylab = NA,
    asp = 1
)
contour(x_coord, y_coord, $recon_h, add = TRUE)
image(
    x = $(collect(hist_msc_12.edges[1])),
    y = $(collect(hist_msc_12.edges[2])),
    w_msc, 
    #useRaster = TRUE, 
    col = pal,
    bty = "n",
    yaxt = "n",    
    xlim = range(x_coord),
    ylim = range(y_coord),
    zlim = c(0, max_dens),
    xlab = expression(PC[1]), ylab = NA,
    asp = 1
)
contour(
    x_coord, y_coord, log($recon_bowen), 
    zlim = c(-2, 2), 
    add = TRUE
)

angle <- 60
text(0.8, 4, "H > LE", adj = c(1, 0.5), srt = angle)
text(1.2, 3, "LE > H", adj = c(0, 0.5), srt = angle)


grid.text("(a)",    0, 1, hjust = 0, vjust = 1)
grid.text("(b)", 0.33, 1, hjust = 0, vjust = 1)
grid.text("(c)", 0.66, 1, hjust = 0, vjust = 1)

dev.off()
"""
show_pdf(
    "http://127.0.0.1:8333/files/results/fig/bowen.pdf",
    width = 900,
    height = 500
)

In [ ]:
v1LE = cube_pca.proj[1, 6]
v2LE = cube_pca.proj[2, 6]
σLE = cube_z_trans.scale[6]
v1H = cube_pca.proj[1, 9]
v2H = cube_pca.proj[2, 9]
σH = cube_z_trans.scale[9]
atand(((v1H/σH + v1LE/σLE)/(v2H/σH + v2LE/σLE)))

In [ ]:
R"library(viridis)"

In [ ]:
R"""
pdf($(joinpath(fig_path, "min_max_maps.pdf")), width = 9, height = 7)

layout(
    matrix(
        c(1, 2,
          3, 4,
          5, 6),
        ncol = 2,
        byrow = TRUE
    )
)

pal <- c(rev(viridis(100)))
asp <- 720 / 1440

par(mar = c(0, 0, 0, 0) , bty = "n")

plot(raster($(cube_min_msc[:, 1:600, 5] |> collect |> missing_to_nan |> permutedims |> x -> Float64.(x))),
     axes = FALSE, col = pal, asp = asp)
plot(raster($(cube_max_msc[:, 1:600, 5] |> collect |> missing_to_nan |> permutedims |> x -> Float64.(x))),
     axes = FALSE, col = pal, asp = asp)

plot(raster($(cube_min_msc[:, 1:600, 6] |> collect |> missing_to_nan |> permutedims |> x -> Float64.(x))),
     axes = FALSE, col = pal, asp = asp)
plot(raster($(cube_max_msc[:, 1:600, 6] |> collect |> missing_to_nan |> permutedims |> x -> Float64.(x))),
     axes = FALSE, col = pal, asp = asp)

plot(raster($(cube_min_msc[:, 1:600, 9] |> collect |> missing_to_nan |> permutedims |> x -> Float64.(x))),
     axes = FALSE, col = pal, asp = asp)
plot(raster($(cube_max_msc[:, 1:600, 9] |> collect |> missing_to_nan |> permutedims |> x -> Float64.(x))),
     axes = FALSE, col = pal, asp = asp)

grid.text("(a)",   0,    1, hjust = 0, vjust = 1)
grid.text("(b)", 0.5,    1, hjust = 0, vjust = 1)
grid.text("(c)",   0, 0.66, hjust = 0, vjust = 1)
grid.text("(d)", 0.5, 0.66, hjust = 0, vjust = 1)
grid.text("(e)",   0, 0.33, hjust = 0, vjust = 1)
grid.text("(f)", 0.5, 0.33, hjust = 0, vjust = 1)

grid.text("MSC min", 1/4, 1, vjust = 1.2)
grid.text("MSC max", 3/4, 1, vjust = 1.2)
grid.text(expression("GPP "           * group("[", gC * m^-2 * day^-1, "]")), 0, 5/6, vjust = 1.2, rot = 90)
grid.text(expression("Latent energy " * group("[", W * m^-2,           "]")), 0, 3/6, vjust = 1.2, rot = 90)
grid.text(expression("Sensible heat " * group("[", W * m^-2,           "]")), 0, 1/6, vjust = 1.2, rot = 90)

dev.off()
"""
show_pdf(
    "http://127.0.0.1:8333/files/results/fig/min_max_maps.pdf",
    width = 700,
    height = 900
)

In [ ]:
lon_ax = getAxis("Lon", pca_no_msc_cube)
lat_ax = getAxis("Lat", pca_no_msc_cube)
t_ax = getAxis("Time", pca_no_msc_cube)
#cutpoint = 900

cutlat1 = 44.875
pc1 = 1
cutpoint1 = axVal2Index(lon_ax, cutlat1)

cutlat2 = -62.625
pc2 = 2
cutpoint2 = axVal2Index(lon_ax, cutlat2)

cutlat3 = 98.5
pc3 = 3
cutpoint3 = axVal2Index(lon_ax, cutlat3)




R"""
pdf($(joinpath(fig_path, "msc_anom.pdf")), width = 14, height = 9)
data(coastsCoarse, package = "rworldmap")

ex_scale_factor <- 0.8
color_bar_rel_height <- 0.28
rel_map_width <- 1
below_plot_spacing <- 0.258
bottom_space <- color_bar_rel_height + below_plot_spacing

elem_col <- "black" #"grey50"

pal1 <- c("#543005", "#8C510A", "#BF812D", "#DFC27D", 
              "#F6E8C3", "#F5F5F5", "#C7EAE5", "#80CDC1", 
              "#35978F", "#01665E", "#003C30")
pal1 <- colorRampPalette(colors = pal1)(100)

pal2 <- rev(c("#67001f", "#b2182b","#d6604d", "#f4a582",
          "#fddbc7", "#f7f7f7", "#d1e5f0","#92c5de",
          "#4393c3", "#2166ac","#053061"))
pal2 <- colorRampPalette(colors = pal2)(100)

pal3 <- c("#7f3b08","#b35806","#e08214","#fdb863",
          "#fee0b6","#f7f7f7","#d8daeb","#b2abd2",
          "#8073ac","#542788","#2d004b")
pal3 <- colorRampPalette(colors = pal3)(100)

r2a <- function(x) {
    x <- as.array(x)
    x <- (x)[, dim(x)[2]:1]
}

label_cex <- 1.3
scale_cex <- 1.1
image_x_min_col <- 200

t_ax   <- $(getAxis("Time", pca_no_msc_cube).values |> collect)
lon_ax <- $(getAxis("Lon",  pca_no_msc_cube).values |> collect .|> Float64)
lat_ax <- $(getAxis("Lat",  pca_no_msc_cube).values |> collect .|> Float64)

asp_man <- 720 / 1440

map1 <- $(pca_no_msc_cube[1, pc1, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
map2 <- $(pca_no_msc_cube[1, pc2, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
map3 <- $(pca_no_msc_cube[1, pc3, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)

time_cutout1_pc1 <- $(pca_no_msc_cube[:, pc1, cutpoint1, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
time_cutout1_pc2 <- $(pca_no_msc_cube[:, pc2, cutpoint1, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
time_cutout1_pc3 <- $(pca_no_msc_cube[:, pc3, cutpoint1, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)

time_cutout2_pc1 <- $(pca_no_msc_cube[:, pc1, cutpoint2, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
time_cutout2_pc2 <- $(pca_no_msc_cube[:, pc2, cutpoint2, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
time_cutout2_pc3 <- $(pca_no_msc_cube[:, pc3, cutpoint2, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)

time_cutout3_pc1 <- $(pca_no_msc_cube[:, pc1, cutpoint3, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
time_cutout3_pc2 <- $(pca_no_msc_cube[:, pc2, cutpoint3, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
time_cutout3_pc3 <- $(pca_no_msc_cube[:, pc3, cutpoint3, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)


ex1 <- max(abs(range(c(map1, time_cutout1_pc1, time_cutout2_pc1, time_cutout3_pc1), na.rm = TRUE)))
ex1 <- c(-ex1, ex1) * ex_scale_factor

ex2 <- max(abs(range(c(map2, time_cutout2_pc1, time_cutout2_pc2, time_cutout3_pc2), na.rm = TRUE)))
ex2 <- c(-ex2, ex2) * ex_scale_factor
 
ex3 <- max(abs(range(c(map3, time_cutout3_pc1, time_cutout3_pc2, time_cutout3_pc3), na.rm = TRUE)))
ex3 <- c(-ex3, ex3) * ex_scale_factor


n_lines_below_second_row <- 5

# layout:
#
#   map_pc1, cutouts: pc1,pc2,pc3,
#   map_pc2, cutouts: pc1,pc2,pc3,
#   map_pc3, cutouts: pc1,pc2,pc3,
#   spacer
#   colourbars: pc1, pc2, pc3

layout(matrix(c(1,1,1,    2,2,2,    3,3,3,    4,4,4, 
                5,5,5,    6,6,6,    7,7,7,    8,8,8,
                9,9,9, 10,10,10, 11,11,11, 12,12,12,
                 0,0,0,0,0,0,0,0,0,0,0,0,
                13,13,13,13, 14,14,14,14, 15,15,15,15), 
              nrow = 5, 
              ncol = 12,
              byrow = TRUE),
       heights = c(1, 1, 1, below_plot_spacing, color_bar_rel_height), 
       widths = c(rep(rel_map_width, 4), 1,1,1,1, 1,1,1,1, 1,1,1,1))

# Plot the first row

upper_mar <- 0.5
left_mar <- 4.5

plot_map <- function(map, cutlat, ex, pal, xax = FALSE, 
                     scalecex = scale_cex, elemcol = elem_col, labelcex = label_cex, 
                     upper_mar = 0.5, left_mar = 4.5, lon_axis = lon_ax, lat_axis = lat_ax) {
    par(mar = c(0, left_mar, upper_mar, 0.1))
    image(
        lon_axis, rev(lat_axis), r2a(map), 
        zlim = ex,
        bty = "n", 
        xaxt = "n", 
        xlab = if (xax) expression(paste("Lon [", degree, "]")) else NA,
        ylab = expression(paste("Lat [", degree, "]")),
        ylim = c(-70, 80),
        las = 1,
        col = pal,
        col.lab = elemcol,
        cex.axis = scalecex,
        cex.lab = labelcex,
        axes = FALSE,
        useRaster = TRUE
    )
    plot(coastsCoarse, add = TRUE, col = "black")
    abline(v = cutlat, col = "red")
    axis(2, col = elemcol, col.axis = elemcol, las = 1, cex.axis = scalecex)
    if(xax)
        axis(1, col = elem_col, col.axis = elem_col, las = 1, cex.axis = scale_cex)
}

plot_cutout <- function(time_cutout, ex, pal, xax = FALSE,
                        time_axis = t_ax, lat_axis = lat_ax, 
                        scalecex = scale_cex, elemcol = elem_col, 
                        labelcex = label_cex) {
    par(mar = c(0, 0, upper_mar, 0.1))
    image(
        seq_along(time_axis), 
        rev(lat_axis), 
        r2a(time_cutout), 
        ylim = c(-70, 80),
        cex.axis = scalecex,
        cex.lab = labelcex,
        xlab = if(xax) "Time [Year]" else NA, 
        xaxt = "n",
        ylab = NA,
        yaxt = "n", col = pal, bty = "n", zlim = ex,
        useRaster = TRUE
    )
    if(xax) {
        par(new = TRUE)
        plot(t_ax, seq_along(t_ax), type = "n", bty = "n", axes = FALSE, xlab = NA, ylab = NA, xaxs = "i")
        axis.Date(1, t_ax,col = elem_col, col.axis = elem_col, las = 1, cex.axis = scale_cex)
    }
}

plot_colorbar <- function(pal, ex, lab, scalecex = scale_cex, labelcex = label_cex, elemcol = elem_col) {
        par(mar = c(4, 5, 0, 5))

        image(matrix(1:100), col = pal,
              axes = FALSE, bty = "n", 
              xlab = lab,
              cex.axis = scalecex,
              cex.lab = labelcex,
                col.lab = elemcol,
              useRaster = TRUE)
        axis(1, 
             at = seq(from =    (ceiling(ex[1]) - ex[1]) / (ex[2] - ex[1]), 
                      to =  1 - (ex[2] - floor(ex[2]))   / (ex[2] - ex[1]), 
                      length.out = 5),
             labels = seq(ceiling(ex[1]), floor(ex[2]), length.out = 5),
             col = elemcol, col.axis = elemcol,
             cex.axis = scalecex)
}

# Plot the first row

plot_map(map1, $cutlat1, ex = ex1, pal = pal1)
plot_cutout(time_cutout1_pc1, ex1, pal1)
plot_cutout(time_cutout1_pc2, ex2, pal2)
plot_cutout(time_cutout1_pc3, ex3, pal3)

# Plot the second row

plot_map(map2, $cutlat2, ex = ex2, pal = pal2)
plot_cutout(time_cutout2_pc1, ex1, pal1)
plot_cutout(time_cutout2_pc2, ex2, pal2)
plot_cutout(time_cutout2_pc3, ex3, pal3)

# Plot the third row

plot_map(map3, $cutlat3, ex = ex3, pal = pal3, xax = TRUE)
plot_cutout(time_cutout3_pc1, ex1, pal1, xax = TRUE)
plot_cutout(time_cutout3_pc2, ex2, pal2, xax = TRUE)
plot_cutout(time_cutout3_pc3, ex3, pal3, xax = TRUE)

# Plot color bars

plot_colorbar(pal1, ex1, expression(lower~productivity %<-% PC[1]~anomaly %->% higher~productivity))
plot_colorbar(pal2, ex2, expression(wetter %<-% PC[2]~anomaly %->% dryer)                          )
plot_colorbar(pal3, ex3, expression(darker %<-% PC[3]~anomaly %->% lighter)                        )

# decorations

grid.text("(a)", 0.04, 1,                                       just = c(-0.3, 1.3))
grid.text("(b)",  1/4, 1,                                       just = c(-0.3, 1.3))
grid.text("(c)",  2/4, 1,                                       just = c(-0.3, 1.3))
grid.text("(d)",  3/4, 1,                                       just = c(-0.3, 1.3))
grid.text("(e)", 0.04, (bottom_space + 2) / (bottom_space + 3), just = c(-0.3, 1.3))
grid.text("(f)",  1/4, (bottom_space + 2) / (bottom_space + 3), just = c(-0.3, 1.3))
grid.text("(g)",  2/4, (bottom_space + 2) / (bottom_space + 3), just = c(-0.3, 1.3))
grid.text("(h)",  3/4, (bottom_space + 2) / (bottom_space + 3), just = c(-0.3, 1.3))
grid.text("(i)", 0.04, (bottom_space + 1) / (bottom_space + 3), just = c(-0.3, 1.3))
grid.text("(j)",  1/4, (bottom_space + 1) / (bottom_space + 3), just = c(-0.3, 1.3))
grid.text("(k)",  2/4, (bottom_space + 1) / (bottom_space + 3), just = c(-0.3, 1.3))
grid.text("(l)",  3/4, (bottom_space + 1) / (bottom_space + 3), just = c(-0.3, 1.3))

grid.lines(1/4,           c(0.2, 0.96),                            gp = gpar(col = elem_col))
grid.lines(2/4,           c(0.2, 0.96),                            gp = gpar(col = elem_col))
grid.lines(3/4,           c(0.2, 0.96),                            gp = gpar(col = elem_col))
grid.lines(c(0.07, 0.97), (bottom_space + 2) / (bottom_space + 3), gp = gpar(col = elem_col))
grid.lines(c(0.07, 0.97), (bottom_space + 1) / (bottom_space + 3), gp = gpar(col = elem_col))

# events

# russian heatwave
xx <- 0.85; yy <- (bottom_space + 2 + 0.86) / (bottom_space + 3)
grid.circle(x = (1:3)/4 + xx/4, y = yy, r = 0.03)
grid.text(label = "Russian Heatwave", x = 1/4 + 0.1, y = yy)

# horn of africa flood
xx <- 0.53; yy <- (bottom_space + 2 + 0.5) / (bottom_space + 3)
grid.circle(x = (1:3)/4 + xx/4, y = yy, r = 0.03)
grid.text(label = "Floods Horn of Africa", x = 1/4 + 0.05, y = yy)

# amazon droughts
xx <- 0.8; yy <- (bottom_space + 1 + 0.45) / (bottom_space + 3)
grid.circle(x = (1:3)/4 + xx/4, y = yy, r = 0.03)
xx <- 0.4
grid.circle(x = (1:3)/4 + xx/4, y = yy, r = 0.03)
grid.text(label = "Droughts Amazon", x = 2/4 + 0.45/4, y = yy + 0.06)

# extreme snow sichuan
xx <- 0.63; yy <- (bottom_space + 0.69) / (bottom_space + 3)
grid.circle(x = (1:3)/4 + xx/4, y = yy, r = 0.03)
grid.text(label = "Extreme Snow China", x = 3/4 + 0.01, y = yy)

dev.off()
"""

show_pdf("http://127.0.0.1:8333/files/results/fig/msc_anom.pdf", width = 900, height = 700)

In [ ]:
cube_pca_online

In [ ]:
pca_no_msc_cube

In [ ]:
##### This is the same as the above figure, but no anomalies, in response to comment referee 2

lon_ax = getAxis("Lon", pca_no_msc_cube)
lat_ax = getAxis("Lat", pca_no_msc_cube)
t_ax = getAxis("Time", pca_no_msc_cube)
#cutpoint = 900

cutlat1 = 44.875
pc1 = 1
cutpoint1 = axVal2Index(lon_ax, cutlat1)

cutlat2 = -62.625
pc2 = 2
cutpoint2 = axVal2Index(lon_ax, cutlat2)

cutlat3 = 98.5
pc3 = 3
cutpoint3 = axVal2Index(lon_ax, cutlat3)




R"""
pdf($(joinpath(fig_path, "comp_time_space.pdf")), width = 14, height = 9)
data(coastsCoarse, package = "rworldmap")

ex_scale_factor <- 0.8
color_bar_rel_height <- 0.28
rel_map_width <- 1
below_plot_spacing <- 0.258
bottom_space <- color_bar_rel_height + below_plot_spacing

elem_col <- "black" #"grey50"

pal1 <- c("#543005", "#8C510A", "#BF812D", "#DFC27D", 
              "#F6E8C3", "#F5F5F5", "#C7EAE5", "#80CDC1", 
              "#35978F", "#01665E", "#003C30")
pal1 <- colorRampPalette(colors = pal1)(100)

pal2 <- rev(c("#67001f", "#b2182b","#d6604d", "#f4a582",
          "#fddbc7", "#f7f7f7", "#d1e5f0","#92c5de",
          "#4393c3", "#2166ac","#053061"))
pal2 <- colorRampPalette(colors = pal2)(100)

pal3 <- c("#7f3b08","#b35806","#e08214","#fdb863",
          "#fee0b6","#f7f7f7","#d8daeb","#b2abd2",
          "#8073ac","#542788","#2d004b")
pal3 <- colorRampPalette(colors = pal3)(100)

r2a <- function(x) {
    x <- as.array(x)
    x <- (x)[, dim(x)[2]:1]
}

label_cex <- 1.3
scale_cex <- 1.1
image_x_min_col <- 200

t_ax   <- $(getAxis("Time", pca_no_msc_cube).values |> collect)
lon_ax <- $(getAxis("Lon",  pca_no_msc_cube).values |> collect .|> Float64)
lat_ax <- $(getAxis("Lat",  pca_no_msc_cube).values |> collect .|> Float64)

asp_man <- 720 / 1440

map1 <- $(cube_pca_online[pc1, :, :, 1] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
map2 <- $(cube_pca_online[pc2, :, :, 1] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
map3 <- $(cube_pca_online[pc3, :, :, 1] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)

time_cutout1_pc1 <- $(cube_pca_online[pc1, cutpoint1, :, :]' |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
time_cutout1_pc2 <- $(cube_pca_online[pc2, cutpoint1, :, :]' |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
time_cutout1_pc3 <- $(cube_pca_online[pc3, cutpoint1, :, :]' |> x -> convert(Array, x) |> missing_to_nan .|> Float64)

time_cutout2_pc1 <- $(cube_pca_online[pc1, cutpoint2, :, :]' |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
time_cutout2_pc2 <- $(cube_pca_online[pc2, cutpoint2, :, :]' |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
time_cutout2_pc3 <- $(cube_pca_online[pc3, cutpoint2, :, :]' |> x -> convert(Array, x) |> missing_to_nan .|> Float64)

time_cutout3_pc1 <- $(cube_pca_online[pc1, cutpoint3, :, :]' |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
time_cutout3_pc2 <- $(cube_pca_online[pc2, cutpoint3, :, :]' |> x -> convert(Array, x) |> missing_to_nan .|> Float64)
time_cutout3_pc3 <- $(cube_pca_online[pc3, cutpoint3, :, :]' |> x -> convert(Array, x) |> missing_to_nan .|> Float64)


ex1 <- max(abs(range(c(map1, time_cutout1_pc1, time_cutout2_pc1, time_cutout3_pc1), na.rm = TRUE)))
ex1 <- c(-ex1, ex1) * ex_scale_factor

ex2 <- max(abs(range(c(map2, time_cutout2_pc1, time_cutout2_pc2, time_cutout3_pc2), na.rm = TRUE)))
ex2 <- c(-ex2, ex2) * ex_scale_factor
 
ex3 <- max(abs(range(c(map3, time_cutout3_pc1, time_cutout3_pc2, time_cutout3_pc3), na.rm = TRUE)))
ex3 <- c(-ex3, ex3) * ex_scale_factor


n_lines_below_second_row <- 5

# layout:
#
#   map_pc1, cutouts: pc1,pc2,pc3,
#   map_pc2, cutouts: pc1,pc2,pc3,
#   map_pc3, cutouts: pc1,pc2,pc3,
#   spacer
#   colourbars: pc1, pc2, pc3

layout(matrix(c(1,1,1,    2,2,2,    3,3,3,    4,4,4, 
                5,5,5,    6,6,6,    7,7,7,    8,8,8,
                9,9,9, 10,10,10, 11,11,11, 12,12,12,
                 0,0,0,0,0,0,0,0,0,0,0,0,
                13,13,13,13, 14,14,14,14, 15,15,15,15), 
              nrow = 5, 
              ncol = 12,
              byrow = TRUE),
       heights = c(1, 1, 1, below_plot_spacing, color_bar_rel_height), 
       widths = c(rep(rel_map_width, 4), 1,1,1,1, 1,1,1,1, 1,1,1,1))

# Plot the first row

upper_mar <- 0.5
left_mar <- 4.5

plot_map <- function(map, cutlat, ex, pal, xax = FALSE, 
                     scalecex = scale_cex, elemcol = elem_col, labelcex = label_cex, 
                     upper_mar = 0.5, left_mar = 4.5, lon_axis = lon_ax, lat_axis = lat_ax) {
    par(mar = c(0, left_mar, upper_mar, 0.1))
    image(
        lon_axis, rev(lat_axis), r2a(map), 
        zlim = ex,
        bty = "n", 
        xaxt = "n", 
        xlab = if (xax) expression(paste("Lon [", degree, "]")) else NA,
        ylab = expression(paste("Lat [", degree, "]")),
        ylim = c(-70, 80),
        las = 1,
        col = pal,
        col.lab = elemcol,
        cex.axis = scalecex,
        cex.lab = labelcex,
        axes = FALSE,
        useRaster = TRUE
    )
    plot(coastsCoarse, add = TRUE, col = "black")
    abline(v = cutlat, col = "red")
    axis(2, col = elemcol, col.axis = elemcol, las = 1, cex.axis = scalecex)
    if(xax)
        axis(1, col = elem_col, col.axis = elem_col, las = 1, cex.axis = scale_cex)
}

plot_cutout <- function(time_cutout, ex, pal, xax = FALSE,
                        time_axis = t_ax, lat_axis = lat_ax, 
                        scalecex = scale_cex, elemcol = elem_col, 
                        labelcex = label_cex) {
    par(mar = c(0, 0, upper_mar, 0.1))
    image(
        seq_along(time_axis), 
        rev(lat_axis), 
        r2a(time_cutout), 
        ylim = c(-70, 80),
        cex.axis = scalecex,
        cex.lab = labelcex,
        xlab = if(xax) "Time [Year]" else NA, 
        xaxt = "n",
        ylab = NA,
        yaxt = "n", col = pal, bty = "n", zlim = ex,
        useRaster = TRUE
    )
    if(xax) {
        par(new = TRUE)
        plot(t_ax, seq_along(t_ax), type = "n", bty = "n", axes = FALSE, xlab = NA, ylab = NA, xaxs = "i")
        axis.Date(1, t_ax,col = elem_col, col.axis = elem_col, las = 1, cex.axis = scale_cex)
    }
}

plot_colorbar <- function(pal, ex, lab, scalecex = scale_cex, labelcex = label_cex, elemcol = elem_col) {
        par(mar = c(4, 5, 0, 5))

        image(matrix(1:100), col = pal,
              axes = FALSE, bty = "n", 
              xlab = lab,
              cex.axis = scalecex,
              cex.lab = labelcex,
                col.lab = elemcol,
              useRaster = TRUE)
        axis(1, 
             at = seq(from =    (ceiling(ex[1]) - ex[1]) / (ex[2] - ex[1]), 
                      to =  1 - (ex[2] - floor(ex[2]))   / (ex[2] - ex[1]), 
                      length.out = 5),
             labels = seq(ceiling(ex[1]), floor(ex[2]), length.out = 5),
             col = elemcol, col.axis = elemcol,
             cex.axis = scalecex)
}

# Plot the first row

plot_map(map1, $cutlat1, ex = ex1, pal = pal1)
plot_cutout(time_cutout1_pc1, ex1, pal1)
plot_cutout(time_cutout1_pc2, ex2, pal2)
plot_cutout(time_cutout1_pc3, ex3, pal3)

# Plot the second row

plot_map(map2, $cutlat2, ex = ex2, pal = pal2)
plot_cutout(time_cutout2_pc1, ex1, pal1)
plot_cutout(time_cutout2_pc2, ex2, pal2)
plot_cutout(time_cutout2_pc3, ex3, pal3)

# Plot the third row

plot_map(map3, $cutlat3, ex = ex3, pal = pal3, xax = TRUE)
plot_cutout(time_cutout3_pc1, ex1, pal1, xax = TRUE)
plot_cutout(time_cutout3_pc2, ex2, pal2, xax = TRUE)
plot_cutout(time_cutout3_pc3, ex3, pal3, xax = TRUE)

# Plot color bars

plot_colorbar(pal1, ex1, expression(low~productivity %<-% PC[1] %->% high~productivity))
plot_colorbar(pal2, ex2, expression(wet %<-% PC[2] %->% dry)                           )
plot_colorbar(pal3, ex3, expression(dark %<-% PC[3] %->% light)                        )

# decorations

grid.text("(a)", 0.04, 1,                                       just = c(-0.3, 1.3))
grid.text("(b)",  1/4, 1,                                       just = c(-0.3, 1.3))
grid.text("(c)",  2/4, 1,                                       just = c(-0.3, 1.3))
grid.text("(d)",  3/4, 1,                                       just = c(-0.3, 1.3))
grid.text("(e)", 0.04, (bottom_space + 2) / (bottom_space + 3), just = c(-0.3, 1.3))
grid.text("(f)",  1/4, (bottom_space + 2) / (bottom_space + 3), just = c(-0.3, 1.3))
grid.text("(g)",  2/4, (bottom_space + 2) / (bottom_space + 3), just = c(-0.3, 1.3))
grid.text("(h)",  3/4, (bottom_space + 2) / (bottom_space + 3), just = c(-0.3, 1.3))
grid.text("(i)", 0.04, (bottom_space + 1) / (bottom_space + 3), just = c(-0.3, 1.3))
grid.text("(j)",  1/4, (bottom_space + 1) / (bottom_space + 3), just = c(-0.3, 1.3))
grid.text("(k)",  2/4, (bottom_space + 1) / (bottom_space + 3), just = c(-0.3, 1.3))
grid.text("(l)",  3/4, (bottom_space + 1) / (bottom_space + 3), just = c(-0.3, 1.3))

grid.lines(1/4,           c(0.2, 0.96),                            gp = gpar(col = elem_col))
grid.lines(2/4,           c(0.2, 0.96),                            gp = gpar(col = elem_col))
grid.lines(3/4,           c(0.2, 0.96),                            gp = gpar(col = elem_col))
grid.lines(c(0.07, 0.97), (bottom_space + 2) / (bottom_space + 3), gp = gpar(col = elem_col))
grid.lines(c(0.07, 0.97), (bottom_space + 1) / (bottom_space + 3), gp = gpar(col = elem_col))


dev.off()
"""

show_pdf("http://127.0.0.1:8333/files/results/fig/comp_time_space.pdf", width = 900, height = 700)